In [17]:
# there is some python code:
import scipy
import typing
import math
import numpy as np
import matplotlib.pyplot as plt
import fractions
from sympy import Matrix, init_printing

import typing
from typing import List, Any, Dict, Union
from dataclasses import dataclass

def isEqual(x, y) -> bool:
  return np.isclose(x, y, rtol=1e-05, atol=1e-08, equal_nan=False)

def toInt(numpyThing) -> int:
    return int(np.rint(numpyThing))

Норму точек можно определить разными способами, но обычно при вычислениях пригождаются следующие нормы:

1) Сферическая

$\lVert x \rVert = \sqrt{(\vec{x}, \vec{x})} = \sqrt{x^2_1 + x^2_2 + \dots + x^2_n}$,

где $(x, y) = x_1 y_1 + x_2 y_2 + \dots + x_n y_n$

2) Октаэдрическая

$\lVert x \rVert = x_1 + x_2 + \dots + x_n$

3) Квадратная

$\lVert x \rVert = \max(x_1, x_2, \dots, x_n)$

Соответственно мне требуется это реализовать

In [18]:
@dataclass
class Point:
  x: float
  y: float

def sphereNorm(a: Point, b: Point) -> float:
  x = a.x - b.x
  y = a.y - b.y
  return math.sqrt(x * x + y * y)

def octNorm(a: Point, b: Point) -> float:
  x = abs(a.x - b.x)
  y = abs(a.y - b.y)
  return x + y

def cubeNorm(a: Point, b: Point) -> float:
  x = a.x - b.x
  y = a.y - b.y
  return max(abs(x), abs(y))

a = Point(0.0, 0.0)
b = Point(2.0, 4.0)

c = Point(a.x - b.x, a.y - b.y)
print(f"oct_norm = {octNorm(a, b)}")
assert(isEqual(octNorm(a, b), abs(c.x) + abs(c.y)))

print(f"sphere_norm = {sphereNorm(a, b)}")
assert(isEqual(math.sqrt(c.x * c.x + c.y * c.y), sphereNorm(a, b)))

print(f"cube_norm = {cubeNorm(a, b)}")
assert(isEqual(max(abs(c.x), abs(c.y)), cubeNorm(a, b)))

oct_norm = 6.0
sphere_norm = 4.47213595499958
cube_norm = 4.0


Решение систем линейных уравнений методом Крамера:

$Ax = b$, где $A$ -- матрица, $x$ и $b$ -- столбцы

где $\vec{x} = \{x_1, x_2, \dots, x_n \}$ -- решение

Легкая теорема для 1 семестра, теперь мне надо это как-нибудь запрогать видимо.

$A_i$ -- это матрица, в которой заменили $i$-тый столбец на столбец $b$.

где $x_i = \frac{\displaystyle \det{A_i}}{\displaystyle \det{A}}$

In [19]:
def compareSolutions(sol: np.ndarray, mysol: List[fractions.Fraction]) -> bool:
    eps: float = 0.000000001
    # может стоит сделать

    for i in range(len(mysol)):
        if (abs(sol[i, 0] - float(mysol[i])) > eps):
            return False

    return True

def fractionsPrint(ansVector: List[fractions.Fraction]) -> None:
    assert(isinstance(ansVector, list))

    for number in ansVector:
        print(number)

def KramersMethod(A: np.ndarray, b: np.ndarray) -> List[fractions.Fraction]:
  d: int = toInt(np.linalg.det(A))
  width: int = toInt(np.shape(A)[1])
  height: int = toInt(np.shape(A)[0])
  sol: List[fractions.Fraction] = list()

  for i in range(width):
    Ai: np.ndarray = np.copy(A)

    for j in range(height):
      Ai[j][i] = b[j]

    sol.append(fractions.Fraction(toInt(np.linalg.det(Ai)), d))

  return sol

In [20]:
size: int = 3

A = np.random.randint(0, 20, size=(size, size))
b = np.random.randint(0, 20, size=(size, 1))
print("Сгенерированная матрица", size, "x", size)
display(Matrix(A))

print("Сгенерированный столбец", size, "x", 1)
display(Matrix(b))

print("Правильное решение СЛАУ:")
right_sol = np.linalg.solve(A, b)
display(Matrix(right_sol))

print("Моё решение СЛАУ:")
another_sol = KramersMethod(A, b)
to_print = np.array(another_sol, dtype=float)
display(Matrix(to_print))
assert(compareSolutions(right_sol, another_sol))


Сгенерированная матрица 3 x 3


Matrix([
[19, 17, 17],
[ 5, 10, 12],
[14,  5,  0]])

Сгенерированный столбец 3 x 1


Matrix([
[ 4],
[ 2],
[17]])

Правильное решение СЛАУ:


Matrix([
[ -2.1255230125523],
[ 9.35146443514643],
[-6.74058577405857]])

Моё решение СЛАУ:


Matrix([
[ -2.1255230125523],
[ 9.35146443514644],
[-6.74058577405858]])

В своих следующих лабораторных работах я буду реализовывать более эффективные методы решения СЛАУ, чем метод Крамера.

Пока что план на лабы такой:

1) Итерационный методы решения СЛАУ

2) Метод Гаусса

3, 4, ...) Подумаем ещё